# ML Model for prediction Bank Churn Customers

# Import Libraries and Data

In [ ]:
import ibis
import pandas as pd
import os

### Import Data from Impala

In [ ]:
hdfs = ibis.hdfs_connect(host=os.environ['HDFS_HOST'], port=50070)

In [ ]:
client_impala = ibis.impala.connect(host=os.environ['IP_IMPALA'], port=21050,hdfs_client=hdfs, user=os.environ['CHANDIMA_LOGIN'], password=os.environ['CHANDIMA_PSWD'],auth_mechanism='PLAIN')

In [ ]:
%%time
requete =client_impala.sql('SELECT * FROM churn_bank.clean_bank')
df = requete.execute()

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.shape

### Check co-relation between the features

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
sns.set(style="whitegrid")

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(df.corr())

# Identify the Distribution of Data

In [ ]:
copy = df.copy()
copy.dropna(inplace = True)
sns.distplot(copy["age"])

In [ ]:
copy = df.copy()
copy.dropna(inplace = True)
sns.distplot(copy["balance"])

In [ ]:
copy = df.copy()
copy.dropna(inplace = True)
sns.distplot(copy["estimatedsalary"])

In [ ]:
copy = df.copy()
copy.dropna(inplace = True)
sns.distplot(copy["tenure"])

In [ ]:
copy = df.copy()
copy.dropna(inplace = True)
sns.distplot(copy["isactivemember"])

Age and Balance are normaly distributed. Tenure and isactivemember are uniformly disributed. These distibutions are useful when to replace null values. Either by mean, median or mode.

# Check NULL Values

In [ ]:
def missingValue(data):
    for col in data.columns:
        nb = round(100*data[col].isnull().sum()/len(data),2)
        print(col + " : " + str(nb)+"%")

In [ ]:
missingValue(df)

The dataframe "df" doesn't contain any null values. Therefor no numerical or categorical imputation. 
If exist, it is mandatory to treat null values by appropriate values.

# Plots and Visualization

### Total No of Customers

In [ ]:
df_8 = df.groupby(['geography'])['customerid'].size().to_frame().reset_index()
df_8.columns = ['Country', 'No_of_Customers']
df_8

### Customer Exited vs Not Exited

In [ ]:
sns.countplot(x="exited", data=df, palette= 'Dark2')#'colorblind')
plt.title("Exited vs Not-Exited")
plt.show()

In [ ]:
# Determine number of Churn Customers in dataset 
fraud = df[df['exited'] == 1] 
valid = df[df['exited'] == 0] 
fraud_fraction = round(100*len(fraud)/float(len(valid)),2) 
print("Churn as percentage" + " : " + str(fraud_fraction)+"%")
print('Churn Customers: {}'.format(len(df[df['exited'] == 1]))) 
print('Valid Customers : {}'.format(len(df[df['exited'] == 0]))) 


### Churn according to Gender

In [ ]:
sns.countplot('gender',data=df,hue='exited', palette=["#9b59b6", "#3498db"]).set_title('Churn with Gender')

### Churn according to Geography

In [ ]:
sns.countplot('geography',data=df,hue='exited', palette =sns.color_palette("bright")).set_title('Churn According to Geography')

### Churn acording to Age Group

In [ ]:
age_cat = df[['age','exited']]

In [ ]:
age_cat.head()

In [ ]:
r = [0, 15, 25, 35, 45, 55,120]
g = ['Children 0 - 15','Teenagers 16-25','Youth 26-35','Adult 36- 45','Mature 46- 55','Old 56+']
age_cat['Age_Category'] = pd.cut(age_cat['age'], bins=r, labels=g)

In [ ]:
df_3 = age_cat.groupby('Age_Category').mean()
df_3.reset_index(inplace=True)
plt.figure(figsize=(6,8))
sns.barplot(x='Age_Category', y='exited', data=df_3,palette="plasma").set_title('Churn vs Age Category')
plt.xticks(plt.xticks()[0],g , rotation=45, ha="right")
plt.tight_layout()
plt.show()

### Active Customers accross Countires

In [ ]:
df_7 = df[df['isactivemember'] == 1].groupby(['geography']).size().reset_index(name='count') 
df_7

In [ ]:
df_8 = df[df['isactivemember'] == 0].groupby(['geography']).size().reset_index(name='count') 
df_8

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo


In [ ]:
pyo.init_notebook_mode()
labels = ["France","Germany", "Spain"]

# Create subplots: use 'domain' type for Pie subplot
figsize=(8, 6)
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=[2591, 1248, 1312], name="Active Customers"),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=[2423, 1261, 1165], name="Not Active Customers"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Active vs Not Active Customers accross Countries",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Active', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='Not-Active', x=0.84, y=0.5, font_size=20, showarrow=False)])
fig.show()

# Feature Engineering

### One-Hot encoding our categorical attributes

In [ ]:
list_cat = ['geography', 'gender']
training_data = pd.get_dummies(df, columns = list_cat, prefix = list_cat)

In [ ]:
training_data.head()

### Feature Rescaling

In [ ]:
# speeding up the calculations in an algorithm
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler = StandardScaler()

In [ ]:
#We need to reshape our data since the Scaler takes in arrays
creditscore_train = np.array(training_data["creditscore"]).reshape(-1, 1)
age_train = np.array(training_data["age"]).reshape(-1, 1)
balance_train = np.array(training_data["balance"]).reshape(-1, 1)
estimatedsalary_train = np.array(training_data["estimatedsalary"]).reshape(-1, 1)

training_data["creditscore"] = scaler.fit_transform(creditscore_train)
training_data["age"] = scaler.fit_transform(age_train)
training_data["balance"] = scaler.fit_transform(balance_train)
training_data["estimatedsalary"] = scaler.fit_transform(estimatedsalary_train)

In [ ]:
training_data.head()

# Split the Data Frame

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(training_data, test_size=0.2)

In [ ]:
X_train = train.drop(['exited','rownumber','customerid','surname'], axis=1).values
y_train = train["exited"].values
X_test = test.drop(['exited','rownumber','customerid','surname'], axis=1).values

In [ ]:
X_train.shape, y_train.shape, X_test.shape

### Import XGBClassifier from xgboost 

We use only one model to prediction

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier

Import libraries to check the accruracy of the model. 
Precision, Recall and F1 score to see how many times you predicted correctly

Precision - Precision is the ratio of correctly predicted positive observations to the total predicted positive observations. 

Recall - Recall is the ratio of correctly predicted positive observations to the all observations in actual class - yes.

F1 score -Weighted average of Precision and Recall.

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

### Find out the best hyperparameters 

In [ ]:
from sklearn.model_selection import GridSearchCV

### Create validation Data Set: to avoid overfitting

In [ ]:
X_training, X_valid, y_training, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:

xg_clf = XGBClassifier()

parameters_xg = {"objective" : ["reg:linear"], "n_estimators" : [5, 10, 15, 20]}

grid_xg = GridSearchCV(xg_clf, parameters_xg, scoring=make_scorer(accuracy_score))
grid_xg.fit(X_training, y_training)

xg_clf = grid_xg.best_estimator_

xg_clf.fit(X_training, y_training)
pred_xg = xg_clf.predict(X_valid)
acc_xg = accuracy_score(y_valid, pred_xg)

print("The Score for XGBoost is: " + str(acc_xg))

In [ ]:
precision_xg = precision_score(y_valid,pred_xg, average='weighted')
print("The precision of xgboost is: " + str(precision_xg))
recall_xg = recall_score(y_valid,pred_xg, average='weighted')
print("The recall of xgboost is: " + str(recall_xg))
F1_xg = f1_score(y_valid,pred_xg, average='weighted')
print("The F1 score of xgboost is: " + str(F1_xg))

### Fit the model on unseen data

In [ ]:
xg_clf.fit(X_train, y_train)

In [ ]:
result_sumbission = xg_clf.predict(X_test)

In [ ]:
y_test = test["exited"].values

In [ ]:
submission2 = pd.DataFrame({
        "Id": test["customerid"],
        "Customer_name":test["surname"],
        "expected_Exited": y_test,
        "predicted_Exited": result_sumbission.round()
    })

submission2 = submission2[['Id','Customer_name','expected_Exited','predicted_Exited']]

In [ ]:
submission2.head(10)

### Check the results with confusion_matrix
 Describe the performance of a classification model

In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
cm = confusion_matrix(y_valid, pred_xg) 
cm

In [ ]:
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax); 

# labels, title and ticks
ax.set_xlabel('Predicted labels')
ax.set_ylabel('Actual labels'); 
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['0', '1'])
ax.yaxis.set_ticklabels(['1', '0'])

### At one glance, Expected and Predicted churn 

In [ ]:
fig, ax =plt.subplots(ncols=2)
sns.countplot(x="expected_Exited", data=submission2, ax =ax[0], palette='plasma')
sns.countplot(x="predicted_Exited", data=submission2, ax =ax[1], palette='plasma')

fig.show()

# Save churn_bank_customers.csv file in to HDFS

### Connect to HDFS

In [ ]:
from hdfs import InsecureClient
client_hdfs = InsecureClient('http://'+os.environ['HDFS_HOST']+':50070', user=os.environ['CHANDIMA_LOGIN'])

### Writing Dataframe to hdfs 

In [ ]:
with client_hdfs.write('/user/chandima.pondapelage/Bank/result_csv/churn_bank_customers.csv',encoding = 'utf-8',overwrite=True) as writer:
    submission2.to_csv(writer)